In [1]:
!pip install llama-index llama-index-llms-huggingface
!pip install llama-index-llms-google-genai llama-index

In [2]:
#!pip install youtube-transcript-api # not working
#!pip install -U yt-dlp # not working
!pip install -U openai-whisper
!pip install llama-index-tools-wikipedia llama-index-tools-tavily-research

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.0/485.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=f5e95a573e192dbbff7f8b51c632061d908bfc630ca9db876a5e205baa5cdc6e
  Stored in directory: /root/.cache/pip/wheels/32/d2/9a/801b5cc5b2a1af2e280089b71c326711a682fc1d50ea29d0ed
Successfully built openai-whisper
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11757 sha256=6a4ae18ab365fd4d6c3061160289bd1317e1a49e974e04d1384a95e6bb520516
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


# Import packages

In [3]:
import pandas as pd
import json
import random
import os
import numpy as np


# Import data

In [4]:
#Before running this cell make sure to add "questions.json" in files

# Load the JSON file
with open("questions.json", "r", encoding="utf-8-sig") as f:
    questions_data = json.load(f)

print(type(questions_data))
print(questions_data[0])

<class 'list'>
{'task_id': '8e867cd7-cff9-4e6c-867a-ff5ddc2550be', 'question': 'How many studio albums were published by Mercedes Sosa between 2000 and 2009 (included)? You can use the latest 2022 version of english wikipedia.', 'Level': '1', 'file_name': ''}


In [5]:
# Select random question
item  = random.choice(questions_data)
item = {'task_id': '99c9cc74-fdc8-46c6-8f8d-3ce2d3bfeea3', 'question': 'Hi, I\'m making a pie but I could use some help with my shopping list. I have everything I need for the crust, but I\'m not sure about the filling. I got the recipe from my friend Aditi, but she left it as a voice memo and the speaker on my phone is buzzing so I can\'t quite make out what she\'s saying. Could you please listen to the recipe and list all of the ingredients that my friend described? I only want the ingredients for the filling, as I have everything I need to make my favorite pie crust. I\'ve attached the recipe as Strawberry pie.mp3.\n\nIn your response, please only list the ingredients, not any measurements. So if the recipe calls for "a pinch of salt" or "two cups of ripe strawberries" the ingredients on the list would be "salt" and "ripe strawberries".\n\nPlease format your response as a comma separated list of ingredients. Also, please alphabetize the ingredients.', 'Level': '1', 'file_name': '99c9cc74-fdc8-46c6-8f8d-3ce2d3bfeea3.mp3'}

print(item)

{'task_id': '99c9cc74-fdc8-46c6-8f8d-3ce2d3bfeea3', 'question': 'Hi, I\'m making a pie but I could use some help with my shopping list. I have everything I need for the crust, but I\'m not sure about the filling. I got the recipe from my friend Aditi, but she left it as a voice memo and the speaker on my phone is buzzing so I can\'t quite make out what she\'s saying. Could you please listen to the recipe and list all of the ingredients that my friend described? I only want the ingredients for the filling, as I have everything I need to make my favorite pie crust. I\'ve attached the recipe as Strawberry pie.mp3.\n\nIn your response, please only list the ingredients, not any measurements. So if the recipe calls for "a pinch of salt" or "two cups of ripe strawberries" the ingredients on the list would be "salt" and "ripe strawberries".\n\nPlease format your response as a comma separated list of ingredients. Also, please alphabetize the ingredients.', 'Level': '1', 'file_name': '99c9cc74-f

In [6]:
question_text = item.get('question')
file_name = item.get("file_name")
file_name = file_name if file_name != "" else None
task_id = item.get("task_id")

print(file_name)

99c9cc74-fdc8-46c6-8f8d-3ce2d3bfeea3.mp3


# Set up LLM

In [11]:
from google import genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=GOOGLE_API_KEY)

In [12]:
from llama_index.llms.google_genai import GoogleGenAI

llm = GoogleGenAI(
        model="models/gemini-2.0-flash-lite",
        api_key=GOOGLE_API_KEY
)

In [9]:
#llm.complete('what is the capital of italy?')

# Set up Llamaindex (tools, agents, prompt)

In [15]:
# calculator tool

def calculate(input: dict) -> dict:
    """Simple calculator function"""

    expression = input['input']

    # Remove any potentially unsafe operations
    if any(unsafe in expression for unsafe in ["import", "exec", "eval", "compile", "open", "__"]):
        return {"error": "Unsafe expression"}

    try:
        # Use a safer approach to evaluate mathematical expressions
        # This is a simplified version - in production you'd want more safeguards
        allowed_symbols = {
            'sqrt': np.sqrt, 'pi': np.pi, 'e': np.e,
            'sin': np.sin, 'cos': np.cos, 'tan': np.tan,
            'log': np.log, 'log10': np.log10, 'exp': np.exp,
            'floor': np.floor, 'ceil': np.ceil, 'abs': abs
        }

        # Replace common math operations with Python syntax
        expression = expression.replace('^', '**')
        result = eval(expression, {"__builtins__": {}}, allowed_symbols)
        return {"result": str(result)}
    except Exception as e:
        return {"error": f"Failed to calculate: {str(e)}"}

In [16]:
calculate({'input': 'sqrt(25)'})

{'result': '5.0'}

In [96]:
# youtube tool --> i cannot make it run for free as of now
'''
from llama_index.core import Document
import subprocess

def get_youtube_transcript(input: dict, cookies: str) -> dict:
    """Fetch transcript from YouTube based on video ID."""

    url = input['file_name']
    video_id = url.split("v=")[-1]

    try:
      # download audio
      command = [
          "yt-dlp",
          "-x",
          "--audio-format", "mp3",
          "-o", f"{video_id}.%(ext)s",
          url,
          "--cookies", cookies,
      ]

      subprocess.run(command)
    except:
      return {'error': f"Failed to download audio: {str(e)}"}
    return get_audio_transcript(f"/content/{video_id}.mp3")


#from youtube_transcript_api import YouTubeTranscriptApi
def get_youtube_transcript(input: dict, cookies: str) -> dict:
    """Fetch transcript from YouTube based on video ID."""

    url = input['file_name']
    video_id = url.split("v=")[-1]

    try:
        ytt_api = YouTubeTranscriptApi()
        fetched_transcript = ytt_api.fetch(video_id)
        text = "\n".join([t["text"] for t in fetched_transcript])
        document = Document(text=text)
        return {'result': document.text}
    except Exception as e:
        return {'error': f"Failed to fetch transcript: {str(e)}"}
''';

In [97]:
get_youtube_transcript({'file_name': 'https://www.youtube.com/watch?v=L1vXCYZAYYM'})

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'error': 'Failed to load audio: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers\n  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)\n  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enab

In [92]:
# audio tool
import whisper
from llama_index.core import Document

def get_audio_transcript(file_path: str) -> dict:
    model = whisper.load_model("base")
    try:
        r = model.transcribe(file_path)
        return {"result": Document(text=r["text"]).text}
    except Exception as e:
        return {"error": str(e)}

In [13]:
get_audio_transcript('/content/99c9cc74-fdc8-46c6-8f8d-3ce2d3bfeea3.mp3')

100%|███████████████████████████████████████| 139M/139M [00:06<00:00, 22.6MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


{'result': ' In a saucepan, combine ripe strawberries, granulated sugar, freshly squeezed lemon juice and cornstarch. Cook the mixture over medium heat, stirring constantly until it thickens to a smooth consistency. Remove from heat and stir in a dash of pure vanilla extract. Allow the strawberry pie feeling to cool before using it as a delicious and fruity filling for your pie crust.'}

In [73]:
# python file execution tool
import io
import contextlib

def run_python_file(file_path: str) -> dict:
    """Safely runs a Python script and returns its final printed numeric output."""

    try:
        python_code = open(file_path).read()

        # Create a buffer to capture stdout
        buffer = io.StringIO()

        # Redirect stdout to the buffer
        with contextlib.redirect_stdout(buffer):
            exec(python_code)

        # Get everything that was printed in the script
        output = buffer.getvalue()
        last_output = output.split('\n')[-2]

        if 'error' in output.lower():
            return {'error': f"Error running script:\n{output}"}

        return {'result': f" Whole output: '{output}'. Final numeric output: {last_output}"}

    except Exception as e:
        return {'error': f"Execution failed: {str(e)}"}

In [74]:
run_python_file('/content/f918266a-b3e0-4914-865d-4faa564f1aef.py')

{'result': " Whole output: 'Working...\nPlease wait patiently...\n0\n'. Final numeric output: '0' "}

In [26]:
# Excel tool

def get_info_from_excel(file_path: str) -> dict:
    """Fetch information from an Excel file."""

    try:
        df = pd.read_excel(file_path)
        text = df.to_markdown()  # Convert DataFrame to Markdown for better readability
        document = Document(text=text)
        return {'result': document.text}
    except Exception as e:
        return {'error': f"Failed to fetch data from Excel: {str(e)}"}

In [27]:
get_info_from_excel('/content/7bd855d8-463d-4ed5-93ca-5fe35145f733.xlsx')

{'result': '|    | Location    |   Burgers |   Hot Dogs |   Salads |   Fries |   Ice Cream |   Soda |\n|---:|:------------|----------:|-----------:|---------:|--------:|------------:|-------:|\n|  0 | Pinebrook   |      1594 |       1999 |     2002 |    2005 |        1977 |   1980 |\n|  1 | Wharvton    |      1983 |       2008 |     2014 |    2015 |        2017 |   2018 |\n|  2 | Sagrada     |      2019 |       2022 |     2022 |    2023 |        2021 |   2019 |\n|  3 | Algrimand   |      1958 |       1971 |     1982 |    1989 |        1998 |   2009 |\n|  4 | Marztep     |      2015 |       2016 |     2018 |    2019 |        2021 |   2022 |\n|  5 | San Cecelia |      2011 |       2010 |     2012 |    2013 |        2015 |   2016 |\n|  6 | Pimento     |      2017 |       1999 |     2001 |    2003 |        1969 |   2967 |\n|  7 | Tinseles    |      1967 |       1969 |     1982 |    1994 |        2005 |   2006 |\n|  8 | Rosdale     |      2007 |       2009 |     2021 |    1989 |        2005

In [16]:
from llama_index.core.tools import FunctionTool

calculator_tool = FunctionTool.from_defaults(
    fn=calculate,
    name="calculator",
    description="A calculator that performs basic arithmetic operations."
)

youtube_tool = FunctionTool.from_defaults(
    fn=get_youtube_transcript,
    name="youtube_video_parser",
    description="A transcript extractor for youtube videos based on video path."
)

audio_tool = FunctionTool.from_defaults(
    fn=get_audio_transcript,
    name="audio_parser",
    description="A simple transcript extractor for audio based on file."
)

run_python_tool = FunctionTool.from_defaults(
    fn=run_python_file,
    name="python_code_executor",
    description="Executes a .py file and returns the final printed numeric result."
)

excel_tool = FunctionTool.from_defaults(
    fn=get_info_from_excel,
    name="excel_parser",
    description="A simple tool to extract information from an Excel file and trasform it into markdown text."
)

In [17]:
created_tools = [calculator_tool, youtube_tool, audio_tool, run_python_tool, excel_tool]
print(len(created_tools))

5


In [18]:
from getpass import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass("Enter your Tavily API key: ")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

Enter your Tavily API key: ··········


In [19]:
from llama_index.tools.tavily_research.base import TavilyToolSpec
from llama_index.tools.wikipedia import WikipediaToolSpec

wikipedia_tool_spec = WikipediaToolSpec()

# to search for information on the web
tavily_tool_spec = TavilyToolSpec(api_key=TAVILY_API_KEY)

To process images I will create another agent that is based on a Vision model as llm

# Run agent on question to test it

In [20]:
#'''
def create_system_prompt_for_main_agent(tools):
    """Create a more descriptive system prompt that explains the available tools"""
    tool_descriptions = "\n".join([
        f"- {tool._metadata.name}: {tool._metadata.description}"
        for tool in tools
    ])

    system_prompt = f"""You're a helpful general AI assistant with the ability to use tools.

You have access to the following tools:
{tool_descriptions}

When a user's request requires using one of these tools:
1. First think through what information you need and which tool would be appropriate
2. Then provide a clear explanation to the user about your approach
3. Finally use the appropriate tool by including the necessary parameters

Important: If a question requires calculation, execution of python code or parsing of youtube video, audios or Excel file, ALWAYS use the appropriate
tool rather than trying to answer from your knowledge. It doesn't matter if  the answer is straightforward, use tools to ensure accuracy and reliability.

Begin."""

    return system_prompt
#''';

In [21]:
system_prompt = create_system_prompt_for_main_agent(created_tools)

In [22]:
print(system_prompt)

You're a helpful general AI assistant with the ability to use tools.

You have access to the following tools:
- calculator: A calculator that performs basic arithmetic operations.
- youtube_video_parser: A transcript extractor for youtube videos based on video path.
- audio_parser: A simple transcript extractor for audio based on file.
- python_code_executor: Executes a .py file and returns the final printed numeric result.
- excel_parser: A simple tool to extract information from an Excel file and trasform it into markdown text.

When a user's request requires using one of these tools:
1. First think through what information you need and which tool would be appropriate
2. Then provide a clear explanation to the user about your approach
3. Finally use the appropriate tool by including the necessary parameters

Important: If a question requires calculation, execution of python code or parsing of youtube video, audios or Excel file, ALWAYS use the appropriate
tool rather than trying to a

In [23]:
from llama_index.core.agent.workflow import ReActAgent

multi_agent = ReActAgent(
    name='multi_functional_agent',
    description="A general AI assistant that can use perform calculation, parse files, and execute code.",
    system_prompt=system_prompt,  # must be string
    tools=created_tools,
    llm=llm,
    verbose=False,
    can_handoff_to=['wikipedia_agent', 'search_agent']
)

In [24]:
def create_system_prompt_for_others(tools):
    """Create a more descriptive system prompt that explains the available tools"""
    tool_descriptions = "\n".join([
        f"- {tool._metadata.name}: {tool._metadata.description}"
        for tool in created_tools
    ])

    system_prompt = f"""You're a helpful general AI assistant with the ability to use tools.

You have access to the following tools:
{tool_descriptions}

When a user's request requires using one of these tools:
1. First think through what information you need and which tool would be appropriate
2. Then provide a clear explanation to the user about your approach
3. Finally use the appropriate tool by including the necessary parameters

Important: ALWAYS use the appropriate tools rather than trying to answer from your knowledge.
It doesn't matter if  the answer is straightforward, use tools to ensure accuracy and reliability.

Begin."""

    return system_prompt

In [25]:
from llama_index.core.agent.workflow import FunctionAgent

wiki_agent = ReActAgent(
    name='wikipedia_agent',
    description="A general AI assistant that can search Wikipedia for information.",
    system_prompt=create_system_prompt_for_others(wikipedia_tool_spec.to_tool_list()),
    tools=wikipedia_tool_spec.to_tool_list(),
    llm=llm,
    verbose=False,
    can_handoff_to=['multi_functional_agent', 'search_agent']
)

search_agent = ReActAgent(
    name='search_agent',
    description="A general AI assistant that can search the web for information.",
    system_prompt=create_system_prompt_for_others(tavily_tool_spec.to_tool_list()),
    tools=tavily_tool_spec.to_tool_list(),
    llm=llm,
    verbose=False,
    can_handoff_to=['multi_functional_agent', 'wiki_agent']
)

In [26]:
from llama_index.core.agent.workflow import AgentWorkflow
import requests

# Create the workflow
workflow = AgentWorkflow(
    agents=[multi_agent, wiki_agent, search_agent],
    root_agent="multi_functional_agent",
    initial_state = {
        'file_path': f'/content/{file_name}',
    }
)

if file_name != None:
  url = f"https://agents-course-unit4-scoring.hf.space/files/{task_id}"
  response = requests.get(url)

  # Save the file
  with open(file_name, "wb") as f:
      f.write(response.content)

  print("Downloaded:", file_name)

Downloaded: 99c9cc74-fdc8-46c6-8f8d-3ce2d3bfeea3.mp3


In [27]:
extract_prompt = f"""
Answer to the following input question using the agents available to you:

{{question}}

If one agent does not have the right tools to answer the question, it can hand off to another agent that has the right tools.
If the question requires a file, you can use the file name provided in the initial state.

Finish your answer with just YOUR FINAL ANSWER (do not ouput <think> or <tool> tags):
- YOUR FINAL ANSWER should be a number OR as few words as possible OR a comma separated list of numbers and/or strings.
- If you are asked for a number, don't use comma to write your number neither use units such as $ or percent sign unless specified otherwise.
- If you are asked for a string, don't use articles, neither abbreviations (e.g. for cities), and write the digits in plain text unless specified otherwise.
- If you are asked for a comma separated list, apply the above rules depending of whether the element to be put in the list is a number or a string.
"""

In [28]:
# define prompt
extract_prompt = extract_prompt.format(question=question_text)

In [29]:
#import nest_asyncio
from llama_index.core.memory import Memory
#nest_asyncio.apply()

# to keep conversation history
memory = Memory.from_defaults(token_limit=40000)

# run agent with thinking process printed
async def main():
  handler = workflow.run(extract_prompt, memory=memory)

  # così vedo come ragiona il LLM
  async for event in handler.stream_events():
      if hasattr(event, "delta"):
          print(event.delta, end="", flush=True)

  result = await handler
  return result

result = await main()

Thought: The current language of the user is: English. I need to use a tool to extract the ingredients from the audio file.
Action: audio_parser
Action Input: {"file_path": "/content/99c9cc74-fdc8-46c6-8f8d-3ce2d3bfeea3.mp3"}


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Thought: I have the transcript from the audio file. Now I need to extract the ingredients and format them as requested.
Action: python_code_executor
Action Input: {'input': "transcript = ' In a saucepan, combine ripe strawberries, granulated sugar, freshly squeezed lemon juice and cornstarch. Cook the mixture over medium heat, stirring constantly until it thickens to a smooth consistency. Remove from heat and stir in a dash of pure vanilla extract. Allow the strawberry pie feeling to cool before using it as a delicious and fruity filling for your pie crust.'\n\ningredients = []\nif 'ripe strawberries' in transcript:\n    ingredients.append('ripe strawberries')\nif 'granulated sugar' in transcript:\n    ingredients.append('granulated sugar')\nif 'freshly squeezed lemon juice' in transcript:\n    ingredients.append('lemon juice')\nif 'cornstarch' in transcript:\n    ingredients.append('cornstarch')\nif 'pure vanilla extract' in transcript:\n    ingredients.append('vanilla extract')\n\nin

In [30]:
print(result.response.blocks[0].text)

cornstarch, granulated sugar, lemon juice, ripe strawberries, vanilla extract
